In [17]:
!pip install transformers accelerate datasets peft bitsandbytes sentencepiece --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.4 MB/s eta 0:00:00:00:0100:01


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 30 14:59:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
from huggingface_hub import login
login()

In [5]:
!git clone https://github.com/aivle-agent/complaint_system_AI.git
%cd complaint_system_AI

!ls

!unzip complain_data.zip -d data

!ls data


Cloning into 'complaint_system_AI'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 12.11 MiB | 40.26 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/complaint_system_AI/complaint_system_AI/complaint_system_AI
 bandit_prompt_system.py   complain_quality_shap.py   README.md
 complain_data.zip	  'Complaint data.zip'
Archive:  complain_data.zip
  inflating: data/중앙행정기관.csv  
  inflating: data/국립아시아문화전당.csv  
  inflating: data/지방행정기관.csv  
  inflating: data/국민신문고.csv  
국립아시아문화전당.csv	지방행정기관.csv  중앙행정기관.csv  국민신문고.csv


In [25]:
import pandas as pd

df = pd.read_csv('data/중앙행정기관.csv')

In [ ]:
# data1.head(1)['consulting_content']

,consulting_content
0,제목 : 어스앵커 굴착심의 건\n\nQ : 아파트 흙막이가시설 공사 중 어스앵커 시...


In [12]:
group_keys = ["source", "consulting_date", "consulting_category", "consulting_content"]

# 2) classification_category를 컬럼으로 피벗
wide = (
    data1.pivot_table(
        index=group_keys,
        columns="classification_category",
        values="classification",
        aggfunc=lambda x: " / ".join(sorted(set(x)))  # 중복 라벨 있으면 합쳐줌
    )
    .reset_index()
)

In [13]:
wide = wide.rename(columns={
    "consulting_content": "complaint_text",
    "상담 주제": "topic",
    "상담 사유": "reason",
    "상담 결과": "outcome",
    "상담 요건": "requirement",
    "상담 내용": "summary",
})

print(wide.columns)
print(wide.head(3))

Index(['source', 'consulting_date', 'consulting_category', 'complaint_text',
       'outcome', 'summary', 'reason', 'requirement', 'topic'],
      dtype='object', name='classification_category')
classification_category source  consulting_date consulting_category  \
0                        고용노동부         20140121                   -   
1                        고용노동부         20140122                   -   
2                        고용노동부         20160218                   -   

classification_category                                     complaint_text  \
0                        Q : 3개월 정착 수당 받고 그 후로는 기본급 없이 인센티브로만 급여를 받아왔는데...   
1                        Q : 저희 사업장은 2013. 12. 31. 이전까지 퇴직금 누진제가 적용이 되었...   
2                        Q : 기타 공공기관 인사 담당자입니다. 저희 기관에서 정규직 직원 채용 시 지원 ...   

classification_category   outcome   summary reason requirement topic  
0                        추가 상담 필요  일반 민원 상담    민원인    단일 요건 민원    안전  
1                           해결 불가  일반 민원 상담    민원인    단일 요건 민원   

In [19]:
import numpy as np
import json
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import transformers
import torch

# 0. LLaMA 파이프라인 초기화

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

llama_pipe = transformers.pipeline(
    "text-generation",
    model=MODEL_ID,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

# 1. 프롬프트 ARM 정의

@dataclass
class PromptArmConfig:
    """
    각 프롬프트 전략(arm)에 대한 메타 정보.
    feature_vector는 이 전략의 성향을 나타내는 임베딩(스타일/목적 가중치 등).
    """
    name: str
    description: str
    feature_vector: np.ndarray  # shape = (d,)

# 2. Logistic Bandit (arm별 w, H + UCB)

class LogisticPromptBandit:
    """
    프롬프트 전략 선택용 logistic bandit.
    - 각 arm별로 logistic 모델 w_a, H_a 유지
    - UCB 스타일로 exploration (mean + beta * uncertainty)
    """
    def __init__(
        self,
        arms: List[PromptArmConfig],
        lambda_reg: float = 1.0,
        eta: float = 0.1,
        beta: float = 1.0,
    ):
        self.arms = arms
        self.n_arms = len(arms)
        self.d = arms[0].feature_vector.shape[0]

        # arm별 parameter, Hessian 근사
        self.w = np.zeros((self.n_arms, self.d))  # w_a
        self.H = [lambda_reg * np.eye(self.d) for _ in range(self.n_arms)]

        self.eta = eta   # step size
        self.beta = beta # exploration 강도

    def _sigmoid(self, z: float) -> float:
        return 1.0 / (1.0 + np.exp(-z))

    def select_arm(self) -> int:
        """
        현재까지의 w, H를 기반으로 어느 프롬프트 전략 arm을 쓸지 선택.
        score_a = (w_a^T x_a) + beta * sqrt( x_a^T H_a^{-1} x_a )
        """
        scores = []
        for a_idx, arm in enumerate(self.arms):
            x = arm.feature_vector  # (d,)
            mean = float(self.w[a_idx].dot(x))

            invH = np.linalg.inv(self.H[a_idx])
            var = float(x.T @ invH @ x)
            ucb = self.beta * np.sqrt(max(var, 1e-12))

            scores.append(mean + ucb)

        chosen = int(np.argmax(scores))
        return chosen

    def update(self, arm_idx: int, reward: float):
        """
        bandit reward(0~1 범위)를 받아 online logistic regression update.
        reward는 verifier에서 나온 스칼라 점수.
        """
        arm = self.arms[arm_idx]
        x = arm.feature_vector
        z = float(self.w[arm_idx].dot(x))
        p = self._sigmoid(z)  # 현재 model이 보는 "성공 확률"

        # logistic loss gradient: -(y - p) * x
        grad = -(reward - p) * x

        # Hessian 근사 업데이트: H_a += x x^T
        self.H[arm_idx] += np.outer(x, x)

        # Online Newton-style step: w <- w - eta * H^{-1} grad
        invH = np.linalg.inv(self.H[arm_idx])
        step = self.eta * (invH @ grad)
        self.w[arm_idx] -= step

    def explain_current_strategy(self) -> List[str]:
        """
        각 arm에 대해 현재 bandit가 보는 "예상 성공도"를 텍스트로 정리.
        """
        lines = []
        for a_idx, arm in enumerate(self.arms):
            x = arm.feature_vector
            z = float(self.w[a_idx].dot(x))
            p = self._sigmoid(z)
            lines.append(
                f"[{arm.name}] "
                f"예상 성공도 ≈ {p:.3f} | "
                f"feature={np.round(x, 2)} | "
                f"설명: {arm.description}"
            )
        return lines

# 3. LLM Generator (Meta-Llama-3 기반)


class LlamaChatGenerator:
    def __init__(self, base_model, lora_path=None):
        from peft import PeftModel

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)
        model = transformers.AutoModelForCausalLM.from_pretrained(
            base_model, torch_dtype=torch.bfloat16, device_map="auto"
        )

        if lora_path:
            model = PeftModel.from_pretrained(model, lora_path)

        self.pipe = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            device_map="auto",
        )

    def generate(self, system_prompt, user_prompt):
        msgs = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        out = self.pipe(
            msgs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
        )[0]["generated_text"]

        # assistant role만 추출
        if isinstance(out, list):
            for m in out:
                if m.get("role") == "assistant":
                    return m["content"]
        return str(out)



def build_prompt_from_arm(
    base_instruction: str,
    complaint_text: str,
    arm: PromptArmConfig,
) -> str:
    """
    arm의 전략 설명을 user 프롬프트에 녹여서 스타일/방향을 제어.
    - base_instruction: system 쪽에 들어가는 역할/전략 기본 설명
    - 여기서 반환하는 건 user_prompt에 들어갈 텍스트
    """
    return (
        f"아래 민원에 대해 답변을 작성하되, 다음 전략 프로필을 따르세요.\n\n"
        f"[전략 이름]\n{arm.name}\n\n"
        f"[전략 설명]\n{arm.description}\n\n"
        f"[민원 내용]\n{complaint_text}\n\n"
        f"[추가 지침]\n"
        f"- 관련 법령/정책과의 일치성을 확인하고, 처리 가능/불가능을 명확히 구분하세요.\n"
        f"- 민원인이 이해하기 쉬운 구조로 답변하고, 필요한 경우 대안/절차를 제시하세요.\n"
        f"- 과도한 약속이나 확정적인 표현은 피하고, '담당 부서의 최종 판단'이 필요함을 밝혀주세요.\n"
    )

# 4. Verifier: LLM 기반 점수 → reward


@dataclass
class VerificationScores:
    """각 품질 지표 스코어 (0~1)."""
    resolution_likelihood: float    # 실제로 해결/처리가 될 가능성
    policy_legal_alignment: float   # 정책/법령·과거 답변과의 일치도
    explanation_clarity: float      # 구조/논리/명료성
    empathy_tone: float             # 민원인 친화적 톤, 갈등 완화
    risk_safety: float              # 기관 입장에서의 안전성 (높을수록 안전)


class SmallVerifier:
    def __init__(self, base_model, lora_path=None):
        from peft import PeftModel

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)
        model = transformers.AutoModelForSequenceClassification.from_pretrained(
            base_model,
            num_labels=1,   # 0~1 점수
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

        if lora_path:
            model = PeftModel.from_pretrained(model, lora_path)

        self.model = model

    def evaluate(self, complaint, answer):
        text = f"[COMPLAINT]\n{complaint}\n\n[ANSWER]\n{answer}"
        inputs = self.tokenizer(text, truncation=True, max_length=2048, return_tensors="pt").to("cuda")

        with torch.no_grad():
            score = torch.sigmoid(self.model(**inputs).logits)[0].item()
        return score


def aggregate_reward(
    scores: VerificationScores,
    weights: Optional[Dict[str, float]] = None,
) -> float:
    """
    reward 구조:
      r = w1 * 해결 가능성
        + w2 * 정책/법률 일치
        + w3 * 명료성
        + w4 * 공감
        + w5 * 안전성
    """
    if weights is None:
        weights = {
            "resolution_likelihood": 0.35,
            "policy_legal_alignment": 0.25,
            "explanation_clarity": 0.20,
            "empathy_tone": 0.10,
            "risk_safety": 0.10,
        }

    s = (
        scores.resolution_likelihood * weights["resolution_likelihood"]
        + scores.policy_legal_alignment * weights["policy_legal_alignment"]
        + scores.explanation_clarity * weights["explanation_clarity"]
        + scores.empathy_tone * weights["empathy_tone"]
        + scores.risk_safety * weights["risk_safety"]
    )

    return float(max(0.0, min(1.0, s)))

# 5. 전체 엔진: bandit → generator → verifier → bandit update

class PromptBanditEngine:
    """
    - bandit이 프롬프트 전략 선택
    - LLM이 답변 생성
    - verifier가 품질 측정 → reward 산출
    - bandit 업데이트
    """
    def __init__(
        self,
        arms: List[PromptArmConfig],
        base_instruction: str,
        llm: LlamaChatGenerator,
        verifier: LlamaVerifier,
    ):
        self.bandit = LogisticPromptBandit(arms=arms)
        self.base_instruction = base_instruction
        self.llm = llm
        self.verifier = verifier

    def step(
        self,
        complaint_text: str,
        reward_weights: Optional[Dict[str, float]] = None,
        extra_context: str = "",
    ) -> Dict[str, Any]:
        """
        민원 1건에 대해:
          1) bandit으로 전략 arm 선택
          2) 해당 전략으로 user_prompt 구성 + LLM 답변
          3) verifier로 평가 + reward
          4) bandit update
          5) 결과/로그 반환
        """
        # 1. arm 선택
        arm_idx = self.bandit.select_arm()
        arm = self.bandit.arms[arm_idx]

        # 2. prompt 구성 + LLM 답변
        user_prompt = build_prompt_from_arm(
            base_instruction=self.base_instruction,
            complaint_text=complaint_text,
            arm=arm,
        )
        answer = self.llm.generate(
            system_prompt=self.base_instruction,
            user_prompt=user_prompt,
        )

        # 3. verifier 평가
        scores = self.verifier.evaluate(
            complaint_text=complaint_text,
            answer_text=answer,
            extra_context=extra_context,
        )
        reward = aggregate_reward(scores, weights=reward_weights)

        # 4. bandit 업데이트
        self.bandit.update(arm_idx=arm_idx, reward=reward)

        # 5. 현재 전략 방향성 요약
        strategy_view = self.bandit.explain_current_strategy()

        return {
            "chosen_arm_idx": arm_idx,
            "chosen_arm_name": arm.name,
            "chosen_arm_description": arm.description,
            "user_prompt": user_prompt,
            "answer": answer,
            "verification_scores": scores,
            "reward": reward,
            "strategy_view": strategy_view,
        }


# 6. to be decided

def build_default_arms() -> List[PromptArmConfig]:
    """
      [법률/정책 중시 정도, 해결책 제안 정도, 공감/톤, 단호함(제한 강조), 근거 제시 강조]
    """
    return [
        PromptArmConfig(
            name="법률-정책 최우선",
            description="법령, 조례, 내부지침과의 일치성을 최우선으로 하고 책임 범위를 분명히 하는 보수적 답변.",
            feature_vector=np.array([0.9, 0.6, 0.4, 0.8, 0.7]),
        ),
        PromptArmConfig(
            name="민원인 공감형",
            description="민원인의 감정과 상황을 충분히 공감하고, 이해하기 쉬운 언어로 절차와 한계를 설명하는 답변.",
            feature_vector=np.array([0.6, 0.7, 0.9, 0.3, 0.5]),
        ),
        PromptArmConfig(
            name="해결책 제안형",
            description="현실적으로 가능한 대안과 행동 옵션을 최대한 많이 제시하는 해결 중심 답변.",
            feature_vector=np.array([0.7, 0.9, 0.7, 0.4, 0.8]),
        ),
    ]


def main():
    arms = build_default_arms()

    generator = LlamaChatGenerator(
        base_model="meta-llama/Meta-Llama-3-8B-Instruct",
        lora_path="./gen_lora"
    )

    verifier = SmallVerifier(
        base_model="microsoft/Phi-3-mini-4k-instruct",
        lora_path="./verifier_lora"
    )

    engine = PromptBanditEngine(
        arms=arms,
        base_instruction=BASE_SYS_PROMPT,
        llm=generator,
        verifier=verifier,
    )

    results = []
    for idx, row in tqdm(wide.iterrows(), total=len(wide)):
        complaint = row["complaint_text"]
        context = row.get("topic", "")

        out = engine.step(complaint, extra_context=context)

        results.append({
            "complaint": complaint,
            "answer": out["answer"],
            "reward": out["reward"],
            "arm": out["chosen_arm_name"]
        })

    df_results = pd.DataFrame(results)
    df_results.to_csv("full_batch_results.csv", index=False)




     

if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at './gen_lora'

In [18]:
!pip install transformers peft accelerate bitsandbytes sentencepiece sentence-transformers scikit-learn --quiet

In [29]:

import os, re, json
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import List, Dict, Any, Optional
from tqdm import tqdm

import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification
)
from peft import PeftModel

from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
import re


#############################################
# 1) 민원 텍스트에서 Q / A 분리
#############################################

def parse_qa(text: str):
    if not isinstance(text, str):
        return "", ""

    # Q: ~~~ A: ~~~ 구조에서 Q, A 각각 분리
    q_match = re.search(r"Q\s*:\s*(.*?)(?:A\s*:|$)", text, flags=re.DOTALL)
    a_match = re.search(r"A\s*:\s*(.*)", text, flags=re.DOTALL)

    q = q_match.group(1).strip() if q_match else ""
    a = a_match.group(1).strip() if a_match else ""
    return q, a

def clean_text(x: Any) -> str:
    if not isinstance(x, str):
        return ""
    x = re.sub(r"\d{2,3}-\d{3,4}-\d{4}", "[TEL]", x)
    x = re.sub(r"\d{6}-\d{7}", "[RRN]", x)
    x = re.sub(r"[가-힣]{2,3}씨", "[NAME]", x)
    return x.strip()

def convert_df_to_qa(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for _, row in df.iterrows():
        content = row.get("consulting_content", "")
        q_raw, a_raw = parse_qa(content)

        q = clean_text(q_raw)
        a = clean_text(a_raw)

        if len(q) < 5 or len(a) < 5:
            continue

        rows.append({
            "question": q,
            "answer": a,
            "source": row.get("source", ""),
            "date": row.get("consulting_date", ""),
            "category": row.get("consulting_category", ""),
            "classification": row.get("classification", "")
        })
    return pd.DataFrame(rows)


# 4) Dataset Builder (generator/verifier 학습용)


def build_dataset(df: pd.DataFrame):
    data = []
    for _, row in df.iterrows():
        q = clean_text(row.get("question", ""))
        a = clean_text(row.get("answer", ""))

        if len(q) < 5 or len(a) < 5:
            continue

        data.append({"question": q, "answer": a})
    return data

# 5) Train/Test Split


def split_dataset(all_data, test_ratio=0.1):
    total = len(all_data)
    test_size = max(1, int(total * test_ratio))
    return all_data[:-test_size], all_data[-test_size:]




# ============================================================
# 2. 프롬프트 전략 ARM 정의


@dataclass
class PromptArmConfig:
    name: str
    description: str
    feature_vector: np.ndarray   


def build_prompt_arms() -> List[PromptArmConfig]:
    return [
        PromptArmConfig(
            name="법률-정책 최우선",
            description="법령·조례·내부지침과의 일치성을 최우선으로 하고, 책임 범위를 명확히 하는 보수적 답변.",
            feature_vector=np.array([0.9, 0.4, 0.5]),
        ),
        PromptArmConfig(
            name="민원인 공감형",
            description="민원인의 감정과 상황을 충분히 공감하고, 이해하기 쉬운 언어로 절차와 한계를 설명하는 답변.",
            feature_vector=np.array([0.6, 0.9, 0.6]),
        ),
        PromptArmConfig(
            name="해결책 제안형",
            description="현실적으로 가능한 대안과 행동 옵션을 적극적으로 제시하는 해결 중심 답변.",
            feature_vector=np.array([0.7, 0.6, 0.9]),
        ),
    ]

# 3. Contextual Logistic Bandit
#    (arm feature + complaint context feature)
# 

class ContextualLogisticPromptBandit:
    """
    Contextual Bandit:
      - 각 arm마다 고정 feature_vector
      - 각 민원마다 context_vec (질문 길이, 구체성, 클러스터 등)
      - 입력 phi = concat(arm_feature, ctx_vec)
    """
    def __init__(
        self,
        arms: List[PromptArmConfig],
        ctx_dim: int,
        lambda_reg: float = 1.0,
        eta: float = 0.1,
        beta: float = 1.0,
    ):
        self.arms = arms
        self.n_arms = len(arms)
        self.d_arm = arms[0].feature_vector.shape[0]
        self.d_ctx = ctx_dim
        self.d = self.d_arm + self.d_ctx

        # arm별 parameter, Hessian 근사
        self.w = np.zeros((self.n_arms, self.d))
        self.H = [lambda_reg * np.eye(self.d) for _ in range(self.n_arms)]

        self.eta = eta
        self.beta = beta

    def _sigmoid(self, z: float) -> float:
        return 1.0 / (1.0 + np.exp(-z))

    def _phi(self, arm_idx: int, ctx_vec: np.ndarray) -> np.ndarray:
        arm_feat = self.arms[arm_idx].feature_vector
        return np.concatenate([arm_feat, ctx_vec])  # (d,)

    def select_arm(self, ctx_vec: np.ndarray) -> int:
        scores = []
        for a_idx in range(self.n_arms):
            x = self._phi(a_idx, ctx_vec)
            mean = float(self.w[a_idx].dot(x))

            invH = np.linalg.inv(self.H[a_idx])
            var = float(x.T @ invH @ x)
            ucb = self.beta * np.sqrt(max(var, 1e-12))

            scores.append(mean + ucb)
        return int(np.argmax(scores))

    def update(self, arm_idx: int, ctx_vec: np.ndarray, reward: float):
        x = self._phi(arm_idx, ctx_vec)
        z = float(self.w[arm_idx].dot(x))
        p = self._sigmoid(z)

        grad = -(reward - p) * x
        self.H[arm_idx] += np.outer(x, x)

        invH = np.linalg.inv(self.H[arm_idx])
        step = self.eta * (invH @ grad)
        self.w[arm_idx] -= step

    def explain_current_strategy(self, ctx_vec: np.ndarray) -> List[str]:
        lines = []
        for a_idx, arm in enumerate(self.arms):
            x = self._phi(a_idx, ctx_vec)
            z = float(self.w[a_idx].dot(x))
            p = self._sigmoid(z)
            lines.append(
                f"[{arm.name}] 예상 성공도 ≈ {p:.3f} | arm_feat={np.round(arm.feature_vector, 2)}"
            )
        return lines


# ============================================================
# 4. Question Refiner (간단 버전; 나중에 LLM으로 교체 가능)
# ============================================================

class SimpleQuestionRefiner:
    """
    지금은 간단히 공백 정리 + 기본 클리닝만 수행.
    나중에 LLM 기반으로 '질문 구조 교정' 로직으로 교체 가능.
    """
    def refine(self, raw_text: str) -> str:
        t = clean_text(raw_text)
        # TODO: 여기에 LLM 기반 재구성 로직을 넣어도 됨.
        return t


# ============================================================
# 5. DBSCAN 기반 민원 유형화 (임베딩 + 클러스터)
# ============================================================

class ComplaintClusterer:
    """
    - SentenceTransformer로 임베딩
    - DBSCAN으로 군집
    - 새로운 민원 → 임베딩 → 가장 유사한 기존 포인트의 클러스터 라벨 사용
    """
    def __init__(self, model_name="jhgan/ko-sroberta-multitask", eps=0.4, min_samples=5):
        self.encoder = SentenceTransformer(model_name)
        self.dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine")
        self.embeddings = None
        self.labels = None

    def fit(self, texts: List[str]):
        self.embeddings = self.encoder.encode(
            texts,
            batch_size=32,
            show_progress_bar=True,
            convert_to_numpy=True,
            normalize_embeddings=True,
        )
        self.dbscan.fit(self.embeddings)
        self.labels = self.dbscan.labels_
        print("클러스터 개수(노이즈 포함):", len(set(self.labels)))

    def predict_cluster(self, text: str) -> int:
        if self.embeddings is None or self.labels is None:
            return -1
        emb = self.encoder.encode([text], convert_to_numpy=True, normalize_embeddings=True)
        sims = cosine_similarity(emb, self.embeddings)[0]
        idx = int(np.argmax(sims))
        return int(self.labels[idx])


# ============================================================
# 6. Context Vector 생성 (질문 특성 + 클러스터)
# ============================================================

def build_context_vector(refined_q: str, cluster_id: int) -> np.ndarray:
    # 길이 (0~1 스케일)
    length = min(len(refined_q) / 500.0, 1.0)
    # 문장 수 (0~1 스케일)
    num_sent = refined_q.count("다.") + refined_q.count(".")
    num_sent = min(num_sent / 10.0, 1.0)
    # 법률 관련 키워드 여부
    has_law = 1.0 if any(k in refined_q for k in ["법", "조항", "제", "시행령"]) else 0.0
    # 클러스터 id normalize (대충 0~1로)
    cluster_norm = 0.0 if cluster_id == -1 else (cluster_id % 20) / 20.0

    return np.array([length, num_sent, has_law, cluster_norm], dtype=float)


# 7. Generator (LLaMA) + Verifier (Phi-3)


class LlamaChatGenerator:
    def __init__(self, model_id="meta-llama/Meta-Llama-3-8B-Instruct"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )
        self.pipe = transformers.pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device_map="auto",
        )

    def generate(self, system_prompt: str, user_prompt: str, max_new_tokens: int = 256):
        msgs = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        out = self.pipe(
            msgs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
        )[0]["generated_text"]
        return str(out)


class PhiMiniVerifier:
    """
    Q/A를 입력으로 받아 0~1 신뢰도 점수 출력.
    (지금은 단일 score지만, 나중에 다중 지표 확장 가능)
    """
    def __init__(self, model_id="microsoft/Phi-3-mini-4k-instruct"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_id,
            num_labels=1,
            torch_dtype=torch.bfloat16,
            device_map="auto",
        )

    def score(self, question: str, answer: str) -> float:
        text = f"[QUESTION]\n{question}\n\n[ANSWER]\n{answer}"
        inputs = self.tokenizer(
            text,
            truncation=True,
            max_length=2048,
            return_tensors="pt",
        ).to("cuda")

        with torch.no_grad():
            logits = self.model(**inputs).logits
            prob = torch.sigmoid(logits)[0].item()
        return float(prob)

# 8. 프롬프트 조립


BASE_SYS_PROMPT = (
    "당신은 한국 공공기관의 민원 보조 AI입니다. "
    "사실과 법령, 정책과의 일치성, 절차 안내를 중시하며, "
    "과도한 약속이나 확정적인 표현은 피해야 합니다."
)

def build_prompt_from_arm(
    complaint_text: str,
    arm: PromptArmConfig,
) -> str:
    return (
        f"아래 민원에 대해 답변을 작성하되, 제시된 전략 프로필을 따르세요.\n\n"
        f"[전략 이름]\n{arm.name}\n\n"
        f"[전략 설명]\n{arm.description}\n\n"
        f"[민원 내용]\n{complaint_text}\n\n"
        f"[추가 지침]\n"
        f"- 관련 법령/정책과의 일치성을 확인하고, 처리 가능/불가능을 명확히 구분하세요.\n"
        f"- 민원인이 이해하기 쉬운 구조(요약 → 근거 → 절차 → 주의사항)로 답변하세요.\n"
        f"- 과도한 약속이나 확정적인 표현은 피하고, "
        f"'실제 처리 여부는 담당 부서의 최종 판단에 따릅니다.'라는 문장을 반드시 포함하세요.\n"
    )


# ============================================================
# 9. 전체 엔진: Refiner → Cluster → Bandit → Generator → Verifier
# ============================================================

class ComplaintEngine:
    def __init__(
        self,
        arms: List[PromptArmConfig],
        generator: LlamaChatGenerator,
        verifier: PhiMiniVerifier,
        clusterer: ComplaintClusterer,
        refiner: SimpleQuestionRefiner,
        ctx_dim: int,
    ):
        self.arms = arms
        self.generator = generator
        self.verifier = verifier
        self.clusterer = clusterer
        self.refiner = refiner
        self.bandit = ContextualLogisticPromptBandit(arms=arms, ctx_dim=ctx_dim)

    def step(self, raw_complaint: str) -> Dict[str, Any]:
        # 1) 질문 교정
        refined_q = self.refiner.refine(raw_complaint)

        # 2) DBSCAN 기반 유형 클러스터
        cluster_id = self.clusterer.predict_cluster(refined_q)

        # 3) Context feature 벡터
        ctx_vec = build_context_vector(refined_q, cluster_id)

        # 4) Bandit으로 arm 선택
        arm_idx = self.bandit.select_arm(ctx_vec)
        arm = self.arms[arm_idx]

        # 5) Generator 호출 프롬프트 생성
        user_prompt = build_prompt_from_arm(refined_q, arm)
        answer = self.generator.generate(BASE_SYS_PROMPT, user_prompt)

        # 6) Verifier 점수 → reward
        reward = self.verifier.score(refined_q, answer)

        # 7) Bandit 업데이트
        self.bandit.update(arm_idx, ctx_vec, reward)

        # 8) 전략 요약 (현재 context 기준 추정)
        strat_view = self.bandit.explain_current_strategy(ctx_vec)

        return {
            "raw_complaint": raw_complaint,
            "refined_complaint": refined_q,
            "cluster_id": cluster_id,
            "context_vec": ctx_vec,
            "chosen_arm_idx": arm_idx,
            "chosen_arm_name": arm.name,
            "answer": answer,
            "reward": reward,
            "strategy_view": strat_view,
        }


# ============================================================
# 10. 배치 평가 (Train/Test 중 Test 파트에 대해)
# ============================================================

def evaluate_batch(engine: ComplaintEngine, test_data, out_csv="batch_results.csv"):
    logs = []
    for item in tqdm(test_data):
        q = item["question"]
        out = engine.step(q)
        logs.append({
            "question": q,
            "refined_question": out["refined_complaint"],
            "cluster_id": out["cluster_id"],
            "arm": out["chosen_arm_name"],
            "score": out["reward"],
            "answer": out["answer"],
        })
    df = pd.DataFrame(logs)
    df.to_csv(out_csv, index=False)
    return df


# ============================================================
# 11. 실시간 서비스용 래퍼
# ============================================================

def ask(engine: ComplaintEngine, complaint: str):
    out = engine.step(complaint)
    print("========================================")
    print("[선택된 전략] ", out["chosen_arm_name"])
    print("[클러스터 ID]", out["cluster_id"])
    print("[컨텍스트 벡터]", np.round(out["context_vec"], 3))
    print("\n[교정된 민원]\n", out["refined_complaint"])
    print("\n[예상 답변 앞부분]\n", out["answer"][:800])
    print("\n[Verifier 기반 신뢰도 score] ", round(out["reward"], 3))
    print("\n[현재 Bandit 전략 뷰]")
    for line in out["strategy_view"]:
        print("  ", line)
    print("========================================")
    return out


# 12. MAIN: 전체 흐름 연결


def main():
    # 0) 원본 CSV 로드
    raw = pd.read_csv('data/중앙행정기관.csv')

    # 1) Q/A 분리된 DF 생성
    qa_df = convert_df_to_qa(raw)
    print("QA rows:", len(qa_df))
    print(qa_df.head(2))

    # 2) build_dataset + train/test split
    all_data = build_dataset(qa_df)
    print("Total usable Q/A pairs:", len(all_data))

    train_data, test_data = split_dataset(all_data, test_ratio=0.1)
    print("Train size:", len(train_data), "Test size:", len(test_data))

    # 3) 클러스터링에 쓸 질문 텍스트
    train_questions = [d["question"] for d in train_data]

    print("Train questions for clustering:", len(train_questions))
    if len(train_questions) == 0:
        raise RuntimeError("No questions available for clustering. Check parsing/filters.")

    # 4) 클러스터러 학습
    clusterer = ComplaintClusterer()
    clusterer.fit(train_questions)

    # 3) 컴포넌트 초기화
    arms = build_prompt_arms()
    generator = LlamaChatGenerator("meta-llama/Meta-Llama-3-8B-Instruct")
    verifier = PhiMiniVerifier("microsoft/Phi-3-mini-4k-instruct")
    refiner = SimpleQuestionRefiner()

    engine = ComplaintEngine(
        arms=arms,
        generator=generator,
        verifier=verifier,
        clusterer=clusterer,
        refiner=refiner,
        ctx_dim=4,   # build_context_vector에서 4차원 벡터 생성
    )

    # 4) Test 일부에 대한 배치 평가
    print("\n Test set 일부(예: 20개)에 대한 평가 실행 중...")
    df_res = evaluate_batch(engine, test_data[:20], out_csv="batch_results_sample.csv")
    print("샘플 결과 saved to batch_results_sample.csv")
    display(df_res.head())

    # 5) 실시간 예시
    print("\n 실시간 예시 민원 입력 테스트")
    _ = ask(engine, "우리 아파트 단지 앞 도로에 불법주차 차량이 많아서 사고 위험이 큽니다. 어떤 조치를 받을 수 있나요?")


if __name__ == "__main__":
    main()


QA rows: 22410
                                            question  \
0  아파트 흙막이가시설 공사 중 어스앵커 시공을 위해 관할구청에 도로점용을 신청하였으나...   
1  아파트 흙막이가시설 공사 중 어스앵커 시공을 위해 관할구청에 도로점용을 신청하였으나...   

                                              answer source      date  \
0  안녕하십니까? 평소 국토 교통행정에 관심과 애정을 가져 주신 점 깊이 감사드리며, ...  국토교통부  20220922   
1  안녕하십니까? 평소 국토 교통행정에 관심과 애정을 가져 주신 점 깊이 감사드리며, ...  국토교통부  20220922   

  category classification  
0    도로관리과       단일 요건 민원  
1    도로관리과  다수기관 복합 민원 상담  
Total usable Q/A pairs: 22410
Train size: 20169 Test size: 2241
Train questions for clustering: 20169


Batches:   0%|          | 0/631 [00:00<?, ?it/s]

클러스터 개수(노이즈 포함): 82


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



 Test set 일부(예: 20개)에 대한 평가 실행 중...


  0%|          | 0/20 [00:12<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 370.12 MiB is free. Process 74025 has 14.38 GiB memory in use. Of the allocated memory 14.21 GiB is allocated by PyTorch, and 44.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)